In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
mnist = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [ ]:
y = mnist.iloc[:, 0].values
X = mnist.iloc[:, 1:].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1
                                                    , random_state = 0)

In [ ]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
nrows = 2
ncols = 4

pic_indexes = np.random.randint(X_train.shape[0], size=(nrows*ncols))

f, axs = plt.subplots(nrows, ncols, figsize=(12, 8))
axs = axs.flatten()

picture_index  = 0

for picture_index, ax in zip(pic_indexes, axs):
        ax.imshow(X_train[picture_index].reshape(28, 28), cmap='gray')
        ax.set_title(f'Label: {y_train[picture_index]}')

plt.show()

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.35),
    
    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer = Adam(lr = 1e-3),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range= 10,
        zoom_range = 0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1
)

datagen.fit(X_train)

In [ ]:
train_generator = datagen.flow(X_train, y_train, batch_size = 64)

validation_generator = datagen.flow(X_test, y_test, batch_size = 64)

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.5, min_lr = 1e-6)

model_checkpoint = ModelCheckpoint('./best_model.hdf5',monitor = 'val_loss', mode = "min", verbose = 1, save_best_model = True)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = X_train.shape[0] // 64,
    epochs = 50,
    validation_data = validation_generator,
    validation_steps = X_test.shape[0] // 64,
    callbacks = [learning_rate_reduction, model_checkpoint]
)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = len(accuracy)

plt.plot(range(epochs), accuracy, 'b', label='Training accuracy')
plt.plot(range(epochs), val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(range(epochs), loss, 'b', label='Training Loss')
plt.plot(range(epochs), val_loss, 'r', label='Training Loss')
plt.title('Training and validation loss')

plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("./best_model.hdf5")

In [ ]:
model.evaluate(X_test, y_test, verbose = 1)

In [ ]:
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv").values
test = test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_pred = model.predict(test).argmax(axis=1)
y_pred.shape

In [ ]:
submission = pd.DataFrame({'ImageId': np.arange(1, 28001), 'Label': y_pred})
submission.to_csv("submission.csv", index = False)
print("Your submission was successfully saved!")